In [1]:
import pymc as pm
import numpy as np
import arviz as az
import aesara.tensor.subtensor as st

%load_ext lab_black
%load_ext watermark

# Insulin

Similar to the Simvstatin example, but with corner constraints.

Adapted from [unit 7: insulin.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/insulin.odc).

No associated lecture.

## Problem statement

Insulin has anti-inflammatory effects, as evaluated by its ability to reduce  plasma concentrations of cytokines. The cytokine content in several organs after endotoxin (lipopolysaccharide, LPS) exposure and the effect of hyperinsulinaemia was examined in a porcine model (Brix-Christensen et al., 2005). All animals (35 to 40 kg) were subject to general anaesthesia and ventilated for 570 minutes.There were two possible interventions:

LPS: Lipopolysaccharide infusion for 180 minutes; and 

HEC: Hyperinsulinemic euglycemic clamp in 570 minutes (from start).

Insulin was infused at a constant rate and plasma glucose was clamped at a certain level by infusion of glucose. LPS induces a systemic inflammation (makes the animals sick) and HEC  acts asa treatment. There were four experimental cells: (1) only anaesthesia (no HEC, no LPS),(2) HEC, (3) LPS, and (4) HEC and LPS.The responses are levels of interleukin-10 (IL-10) in the kidney after 330 minutes have elapsed.


In [2]:
# fmt: off
IL = np.array(
              (7.0607, 2.6168, 4.3489, 3.6356, 4.7510, 2.9530, 3.6137, 5.6969,
               3.0693, 1.6489, 2.9160, 2.9149, 2.1102, 3.1004, 4.1170, 3.0229, 
               3.6911, 4.3933, 6.0513, 4.2559, 4.5554, 3.8447, 1.3590, 2.1449, 
               2.4159, 3.1493, 4.4462, 2.8545, 1.8944, 3.5133, 4.6254, 3.8967)
             )

hec = np.array(
                (1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 
                 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2)
              )
lps  = np.array(
                (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2)
               )
# fmt: on

In [3]:
coords = {"hec": np.unique(hec), "id": np.array(range(len(lps))), "lps": np.unique(lps)}

In [4]:
coords

{'hec': array([1, 2]),
 'id': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 'lps': array([1, 2])}

In [5]:
hec_idx = hec - 1
lps_idx = lps - 1

In [6]:
with pm.Model(coords=coords) as m:
    IL_data = pm.Data("apa_data", IL, mutable=False)
    hec_data = pm.Data("hec_data", hec_idx, dims="id", mutable=False)
    lps_data = pm.Data("lps_data", lps_idx, dims="id", mutable=False)

    mu0 = pm.Normal("mu0", 0, tau=0.0001)
    _alpha = pm.Normal("_alpha", 0, tau=0.0001, dims="hec")
    _beta = pm.Normal("_beta", 0, tau=0.0001, dims="lps")
    _alphabeta = pm.Normal("_alphabeta", 0, tau=0.0001, dims=("hec", "lps"))
    tau = pm.Gamma("tau", 0.001, 0.001)
    sigma = pm.Deterministic("sigma", 1 / tau**0.5)

    # sum-to-zero constraints
    alpha = pm.Deterministic("alpha", st.set_subtensor(_alpha[0], 0))
    beta = pm.Deterministic("beta", st.set_subtensor(_beta[0], 0))
    _alphabeta = st.set_subtensor(_alphabeta[:, 0], 0)
    alphabeta = pm.Deterministic("alphabeta", st.set_subtensor(_alphabeta[0, :], 0))

    mu = mu0 + alpha[hec_data] + beta[lps_data] + alphabeta[hec_data, lps_data]
    pm.Normal("IL", mu, tau=tau, observed=IL_data)

    trace = pm.sample(2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu0, _alpha, _beta, _alphabeta, tau]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 64 seconds.


In [7]:
az.summary(
    trace,
    var_names=["~_"],
    filter_vars="like",
    kind="stats",
).loc[["alpha[1]", "mu0", "sigma", "beta[1]", "alphabeta[1, 1]"], :]

,mean,sd,hdi_3%,hdi_97%
alpha[1],-1.470,0.624,-2.674,-0.325
mu0,4.333,0.441,3.516,5.173
sigma,1.234,0.171,0.935,1.560
beta[1],-0.543,0.621,-1.707,0.614
"alphabeta[1, 1]",1.036,0.874,-0.522,2.778


In [ ]:
%watermark -n -u -v -iv -p aesara,aeppl